# BiFlowLISA Main Notebook
This notebook runs the bivariate flow LISA analysis using `execBIFLOWLISA`.

In [ ]:
# If using Colab, uncomment and run:
# !pip install numpy pyshp pandas

import numpy as np
import pandas as pd
import shapefile
import random
from core.getFlowNeighbors import getFlowNeighborsContiguity
from core.spatstats import calculateBivariteMoranI
from core.FlowLISA import execBIFLOWLISA


In [ ]:
# Load flow data
flowdf1 = pd.read_csv('input/Flow37xLL.txt', sep='\s+')
flowdf2 = pd.read_csv('input/Flow37yMM.txt', sep='\s+')

# Build bivariate FlowValue
FlowValue = {}
for (o,d), v1 in zip(zip(flowdf1['O'], flowdf1['D']), flowdf1['Flow']):
    FlowValue[(o,d)] = [v1, None]
for (o,d), v2 in zip(zip(flowdf2['O'], flowdf2['D']), flowdf2['Flow']):
    if (o,d) in FlowValue:
        FlowValue[(o,d)][1] = v2
    else:
        FlowValue[(o,d)] = [None, v2]

print(f'Loaded {len(FlowValue)} flow pairs')

In [ ]:
# Read origin/destination shapefiles
sf1 = shapefile.Reader('input/Hex37_O.shp')
sf2 = shapefile.Reader('input/Hex37_D.shp')
shapes1 = sf1.shapes()
shapes2 = sf2.shapes()

# Prepare AREAS lists
AREAS1 = [[geom.points] for geom in shapes1]
AREAS2 = [[geom.points] for geom in shapes2]


In [ ]:
# Run the BiFlowLISA analysis
output = execBIFLOWLISA(AREAS1, AREAS2, FlowValue, NeiLvl=120)
print(output)

In [ ]:
# Save results to file
import os
os.makedirs('result', exist_ok=True)
with open('result/BiFlowLISA_output.txt', 'w') as f:
    f.write(output)
print('Results saved to result/BiFlowLISA_output.txt')